# Forms e Validação

## Oque é o Forms?

Quando estamos desenvolvendo uma aplicação web, é comum permitir que os usuários enviem informações através de formulários. O Django facilita a criação e gerenciamento de formulários, tornando o processo mais seguro e eficiente.

Os forms do Django são uma parte fundamental do framework, permitindo que você manipule dados de entrada do usuário de maneira eficiente e segura. Os forms no Django ajudam na validação e renderização de formulários HTML, além de permitir a fácil manipulação de dados provenientes de requisições.

### Principais Características dos Forms do Django

**Validação Automática**:O Django oferece validação automática dos dados enviados através de formulários. Você pode definir regras de validação e, se os dados não atenderem a essas regras, o Django retorna mensagens de erro informativas.

**Renderização Simples**:Com a ajuda dos forms do Django, você pode renderizar rapidamente os campos de entrada no HTML. Isso facilita a criação de interfaces de usuário para coleta de dados.

**Integração com Modelos**:Os forms do Django podem ser facilmente integrados com modelos do Django, permitindo que você crie formulários a partir de um modelo existente. Isso é feito através do uso de ModelForm.

**Segurança**:O Django protege automaticamente seus formulários contra ataques CSRF (Cross-Site Request Forgery) e XSS (Cross-Site Scripting).
Estilo Personalizado:

Você pode personalizar a aparência dos formulários, definindo classes CSS, atributos HTML e outros estilos diretamente nos campos do formulário.

### Estrutura do Projeto

In [ ]:
myproject/
├── myapp/
│   ├── migrations/
│   ├── templates/
│   │   ├── cadastrar_produto.html
│   │   ├── sucesso.html
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── forms.py  # Aqui está o seu formulário
│   ├── models.py  # Aqui está o seu modelo Produto
│   ├── tests.py
│   ├── urls.py  # Aqui adicionamos a URL do formulário
│   └── views.py  # Aqui está a sua view de cadastro

### Criando formulários com Django Forms

Em vez de criar manualmente todos os campos do formulário, podemos usar um ModelForm para criar formulários diretamente a partir de um modelo Django. Isso facilita o trabalho e reduz a duplicação de código.

In [ ]:
from django.forms import ModelForm
from .models import Produto

class ProdutoModelForm(ModelForm):
    class Meta:
        model = Produto
        fields = ['nome', 'preco', 'estoque']

Isso gera automaticamente o formulário com base nos campos do modelo Produto.

### Processamento de formulários 

Quando lidamos com formulários, é importante entender o ciclo de requisição HTTP. Ao acessar a página pela primeira vez, a requisição é via GET. Quando o usuário submete o formulário, a requisição é POST.

In [ ]:
@csrf_exempt  # Exclui a verificação de token CSRF (use com cautela)
def criar_produto(request):
    if request.method == 'POST':
        # Instancia o formulário com os dados recebidos na requisição POST
        form = ProdutoForm(request.POST)
        
        # Verifica se os dados do formulário são válidos
        if form.is_valid():
            # Salva o produto no banco de dados
            produto = form.save()
           
    else:
        form = ProdutoForm()  # Cria um formulário vazio para GET
    
    # Renderiza o template com o formulário
    return render(request, 'criar_produto.html', {'form': form})

Exibindo o ModelForm na View

In [ ]:
@csrf_exempt  # Exclui a verificação de token CSRF (use com cautela)
def criar_produto(request):
    if request.method == 'POST':
        # Instancia o formulário com os dados recebidos na requisição POST
        form = ProdutoForm(request.POST)
        
        # Verifica se os dados do formulário são válidos
        if form.is_valid():
            # Salva o produto no banco de dados
            produto = form.save()
            # Redireciona para uma página de sucesso ou outra view
            return redirect('produto_sucesso')  # Altere para a URL de redirecionamento desejada
    else:
        form = ProdutoForm()  # Cria um formulário vazio para GET
    
    # Renderiza o template com o formulário
    return render(request, 'criar_produto.html', {'form': form})

### renderizando no template

## O que é Validação?

Validação é o processo de garantir que os dados fornecidos pelo usuário em um formulário atendam a certos critérios ou regras definidas antes de serem processados e armazenados. Esse processo é fundamental para assegurar a integridade, a segurança e a qualidade dos dados em uma aplicação. 

Em Django, a validação pode ser feita de várias formas, incluindo:

**Validação de Campo**: Cada campo em um formulário pode ter suas próprias regras de validação. Por exemplo, um campo de email deve conter um formato de email válido, e um campo de preço deve ser um número positivo.

**Validação Personalizada**: Além das validações padrão fornecidas pelo Django, você pode implementar suas próprias regras de validação, permitindo que sua aplicação atenda a requisitos de negócios específicos.

**Validação de Formulário**: Além de validar campos individuais, você pode verificar a consistência de dados em todo o formulário. Por exemplo, você pode querer garantir que duas senhas inseridas sejam idênticas.

### Importância da Validação

**Integridade dos Dados**: Validações ajudam a garantir que apenas dados válidos e consistentes sejam armazenados no banco de dados, o que é essencial para a integridade das informações.

**Segurança**: A validação ajuda a prevenir ataques como SQL Injection, Cross-Site Scripting (XSS) e outras vulnerabilidades de segurança. Por exemplo, validando que um campo não contenha scripts maliciosos.

**Experiência do Usuário**: Quando os dados são validados corretamente, os usuários recebem feedback imediato sobre erros em suas entradas. Isso melhora a experiência do usuário, pois eles podem corrigir problemas antes que o formulário seja enviado.

**Redução de Erros**: Com a validação, você pode evitar erros comuns, como números negativos em campos que devem ser positivos ou strings vazias em campos obrigatórios.

**Facilidade de Manutenção**: Aplicações bem validadas são mais fáceis de manter e evoluir ao longo do tempo, uma vez que os requisitos de dados são claros e respeitados.

### Atualizando o Formulário com Validação Personalizada

Você pode adicionar validação personalizada diretamente no seu formulário Django. Abaixo, adicionamos validação para garantir que o preço do produto seja positivo e que o estoque não seja negativo.

In [ ]:
class ProdutoForm(forms.ModelForm):
    class Meta:
        model = Produto
        fields = ['nome', 'descricao', 'preco', 'estoque']

    # Validação personalizada para o campo 'preco'
    def clean_preco(self):
        preco = self.cleaned_data.get('preco')
        if preco <= 0:
            raise forms.ValidationError("O preço deve ser um valor positivo.")
        return preco

    # Validação personalizada para o campo 'estoque'
    def clean_estoque(self):
        estoque = self.cleaned_data.get('estoque')
        if estoque < 0:
            raise forms.ValidationError("O estoque não pode ser negativo.")
        return estoque

### Atualizando a View para Lidar com Erros

Agora, vamos garantir que nossa view criar_produto esteja configurada para lidar com erros de validação e exibir mensagens apropriadas.

In [ ]:
@csrf_exempt  # Exclui a verificação de token CSRF (use com cautela)
def criar_produto(request):
    if request.method == 'POST':
        # Instancia o formulário com os dados recebidos na requisição POST
        form = ProdutoForm(request.POST)
        
        # Verifica se os dados do formulário são válidos
        if form.is_valid():
            # Salva o produto no banco de dados
            produto = form.save()
            # Redireciona para uma página de sucesso ou outra view
            return redirect('produto_sucesso')  # Altere para a URL de redirecionamento desejada
        else:
            # O formulário não é válido, então ele será renderizado novamente com os erros
            return render(request, 'criar_produto.html', {'form': form})
    else:
        form = ProdutoForm()  # Cria um formulário vazio para GET
    
    # Renderiza o template com o formulário
    return render(request, 'criar_produto.html', {'form': form})

Exibindo Mensagens de Erro no Template

No template criar_produto.html, já adicionamos a lógica para exibir mensagens de erro. Vamos garantir que esta parte do código esteja clara:

#